# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv("spam.csv")

spam_data["target"] = np.where(spam_data["target"] == "spam", 1, 0)
spam_data.head()

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(spam_data["text"], 
                                                   spam_data["target"],
                                                   random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    ratio = np.mean(spam_data["target"])
    
    return ratio*100

In [4]:
answer_one()

13.406317300789663

In [5]:
# Alternatively
def answer_one():
    ratio = len(spam_data[spam_data["target"] == 1]) / len(spam_data)
    return ratio * 100
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train).vocabulary_
    
    vect = [x for x in vect.keys()]
    len_vect = [len(x) for x in vect]
    result = vect[np.argmax(len_vect)]
    
    return result

In [7]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    # Using MultinomialNB
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predictions = model.predict(X_test_vectorized)
    result = float(roc_auc_score(y_test, predictions))
    
    
    return result

In [9]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
    tfidf = X_train_vectorized.max(0).toarray()[0]
    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    sorted_tfidf = tfidf[sorted_tfidf_index]
    
    smallest_tfidf_series = pd.Series(sorted_tfidf[:20], index=feature_names[sorted_tfidf_index[:20]])
    largest_tfidf_series = pd.Series(sorted_tfidf[-21:][::-1], index=feature_names[sorted_tfidf_index[-21:][::-1]])
    
    return (smallest_tfidf_series, largest_tfidf_series)

In [32]:
answer_four()

(sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702
 sam

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    # Using MultinomialNB
    model = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predictions = model.predict(X_test_vectorized)
    result = float(roc_auc_score(y_test, predictions))
    
    return result

In [13]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [14]:
def answer_six():
    len_spam = [len(x) for x in spam_data.loc[spam_data["target"] == 1, "text"]]
    len_not_spam = [len(x) for x in spam_data.loc[spam_data["target"] == 0, "text"]]
    average_spam = np.mean(len_spam)
    average_not_spam = np.mean(len_not_spam)
    
    return (average_not_spam, average_spam)

In [15]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [16]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    len_train = [len(x) for x in X_train]
    len_test = [len(x) for x in X_test]
    
    new_train = add_feature(X_train_vectorized, len_train)
    new_test = add_feature(X_test_vectorized, len_test)
    
    # Using SVM
    model = SVC(C=10000).fit(new_train, y_train)
    predictions = model.predict(new_test)
    result = roc_auc_score(y_test, predictions)
    
    return float(result)

In [18]:
answer_seven()

0.9581366823421557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [21]:
def answer_eight():
    digit_spam = [sum(char.isnumeric() for char in x) for x in spam_data.loc[spam_data["target"] == 1, "text"]]
    digit_not_spam = [sum(char.isnumeric() for char in x) for x in spam_data.loc[spam_data['target'] == 0, "text"]]
    average_spam = np.mean(digit_spam)
    average_not_spam = np.mean(digit_not_spam)
    
    return (average_not_spam, average_spam)

In [22]:
answer_eight()

(0.29927461139896372, 15.76037483266399)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [33]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vec = TfidfVectorizer(min_df=5, ngram_range=(1, 3)).fit(X_train)
    X_train_vectorized = vec.transform(X_train)
    X_test_vectorized = vec.transform(X_test)
    
    # Calculating the number of digits in each document
    train = [len(x) for x in X_train]
    test = [len(x) for x in X_test]
    digits_train = [sum(char.isnumeric() for char in x) for x in X_train]
    digits_test = [sum(char.isnumeric() for char in x) for x in X_test]
    
    # Creating new_train, and new_test
    new_train = add_feature(X_train_vectorized, [train, digits_train])
    new_test = add_feature(X_test_vectorized, [test, digits_test])
    
    # Creating the model
    model = LogisticRegression(C=100).fit(new_train, y_train)
    predictions = model.predict(new_test)
    result = float(roc_auc_score(y_test, predictions))
    
    return result

In [34]:
answer_nine()

0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [27]:
def answer_ten():
    spam = np.mean(spam_data.loc[spam_data["target"] == 1, "text"].str.count("\W"))
    not_spam = np.mean(spam_data.loc[spam_data["target"] == 0, "text"].str.count("\W"))
    
    return (not_spam, spam)

In [28]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [38]:
def answer_eleven():
    # Using CountVectorizer
    vect = CountVectorizer(min_df=5, ngram_range=(2, 5), analyzer="char_wb").fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    # Finding the length of document
    train_length = [len(x) for x in X_train]
    test_length = [len(x) for x in X_test]
    
    # Finding number of digits per document
    train_digits = [sum(char.isnumeric() for char in x) for x in X_train]
    test_digits = [sum(char.isnumeric() for char in x) for x in X_test]
    
    # Finding non_word characters 
    train_char = X_train.str.count("\W")
    test_char = X_test.str.count("\W")
    
    # Creating new train_data and test_data
    new_train = add_feature(X_train_vectorized, [train_length, train_digits, train_char])
    new_test = add_feature(X_test_vectorized, [test_length, test_digits, test_char])
    
    # Creating the model
    model = LogisticRegression(C=100).fit(new_train, y_train)
    predictions = model.predict(new_test)
    AUC = float(roc_auc_score(y_test, predictions))
    
    # Getting the Feature names
    feature_names = np.array(vect.get_feature_names() + ["length_of_doc", "digit_count", "non_word_char_count"])
    sorted_coef_index = model.coef_[0].argsort()
    smallest_coef = list(feature_names[sorted_coef_index[:10]])
    largest_coef = list(feature_names[sorted_coef_index[-11:][::-1]])
    
    return (AUC, smallest_coef, largest_coef)

In [39]:
answer_eleven()

(0.9788593110707434,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count',
  'ne',
  'ia',
  'co',
  'xt',
  ' ch',
  'mob',
  ' x',
  'ww',
  'ar',
  'eply '])